In [1]:
import pickle
import gurobipy as gp
from gurobipy import GRB

In [3]:
with open('../data/B.pickle', 'rb') as handle:
    B = pickle.load(handle)
print(B)
with open('../data/R.pickle', 'rb') as handle:
    R = pickle.load(handle)
print(R)

{0: (0, [300, 155, 2, 140, 0, 0, 0]), 1: (0, [300, 155, 2, 140, 0, 0, 0]), 2: (1, [192, 155, 2, 200, 0, 0, 0]), 3: (1, [192, 155, 2, 200, 0, 0, 0])}
{0: (98, 50, 1, 0, 0, 0), 1: (89, 46, 1, 0, 0, 0), 2: (64, 24, 1, 0, 0, 0), 3: (93, 29, 1, 1, 0, 0), 4: (114, 54, 1, 1, 0, 0), 5: (95, 60, 1, 0, 0, 0), 6: (63, 28, 1, 0, 0, 0), 7: (100, 46, 0, 0, 0, 0), 8: (52, 61, 1, 0, 0, 0), 9: (45, 46, 1, 0, 0, 0), 10: (111, 32, 1, 0, 0, 0), 11: (109, 38, 1, 0, 1, 0), 12: (97, 57, 1, 0, 0, 0), 13: (51, 29, 1, 0, 0, 0), 14: (86, 54, 1, 0, 0, 0), 15: (114, 31, 1, 1, 0, 0), 16: (81, 47, 1, 0, 0, 0), 17: (78, 25, 1, 0, 0, 0), 18: (78, 44, 1, 0, 0, 0), 19: (68, 33, 1, 0, 0, 0), 20: (51, 45, 1, 0, 0, 0), 21: (84, 36, 1, 0, 0, 1), 22: (66, 35, 0, 0, 0, 0), 23: (68, 42, 1, 1, 0, 0), 24: (108, 57, 1, 1, 1, 0)}


In [5]:
#Define parameters
n = len(R) #total boxes to be packed
m = len(B) #total ULDs available


In [7]:
try:
    # Create a new model
    m = gp.Model("mip")

    # Create variables
    p_ij = m.addVar(vtype=GRB.BINARY, name="p_ij") #if box i is in container j
    u_j = m.addVar(vtype=GRB.BINARY, name="u_j") #if container j is used
    x_i = m.addVar(vtype=GRB.INTEGER, name="x_i") #bottom left corner of box i
    z_i = m.addVar(vtype=GRB.INTEGER, name="z_i") #bottom left corner of box i
    x_i_ = m.addVar(vtype=GRB.INTEGER, name="x_i_") #top right corner of box i
    z_i_ = m.addVar(vtype=GRB.INTEGER, name="z_i_") #top right corner of box i
    r_iab = m.addVar(vtype=GRB.BINARY, name="r_iab") #if side b of box i is along axis a
    xp_ik = m.addVar(vtype=GRB.BINARY, name="xp_ik") #if box i is on right of box k
    zp_ik = m.addVar(vtype=GRB.BINARY, name="zp_ik") #if box i is above box k
    r_i11 = m.addVar(vtype=GRB.BINARY, name="r_i11") #if length is along x axis
    r_i13 = m.addVar(vtype=GRB.BINARY, name="r_i13") #if height is along x axis
    r_i31 = m.addVar(vtype=GRB.BINARY, name="r_i31") #if length is along z axis
    r_i33 = m.addVar(vtype=GRB.BINARY, name="r_i33") #if height is along z axis

    # Set objective
    m.setObjective(u_j * C_j, GRB.MINIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

NameError: name 'C_j' is not defined

In [ ]:
help(m.addVar)

Help on method addVar in module gurobipy:

addVar(lb=0.0, ub=1e+100, obj=0.0, vtype='C', name='', column=None) method of gurobipy.Model instance
    ROUTINE:
      addVar(lb, ub, obj, vtype, name, column)
    
    PURPOSE:
      Add a variable to the model.
    
    ARGUMENTS:
      lb (float): Lower bound (default is zero)
      ub (float): Upper bound (default is infinite)
      obj (float): Objective coefficient (default is zero)
      vtype (string): Variable type (default is GRB.CONTINUOUS)
      name (string): Variable name (default is no name)
      column (Column): Initial coefficients for column (default is None)
    
    RETURN VALUE:
      The created Var object.
    
    EXAMPLE:
      v = model.addVar(ub=2.0, name="NewVar")

